<a href="https://colab.research.google.com/github/DreamXvan/Colab-pytorch-public/blob/main/Just_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
print(torch.__version__)

2.6.0+cu124


In [3]:
print(torch.cuda.is_available())

True


In [1]:
!nvidia-smi

Fri May 16 14:16:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [7]:
import argparse

import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, Subset, DataLoader
from torchvision import datasets, transforms

parser = argparse.ArgumentParser()
parser.add_argument("--device", type=str, default="cuda:3")
parser.add_argument('--lr_net', type=float, default=0.01, help='learning rate for updating network parameters')
parser.add_argument('--batch_train', type=int, default=256, help='batch size for training networks')
parser.add_argument('--epochs', type=int, default=20, help='number of epochs')
args = parser.parse_args()

data_path = 'data'


class ConvNet(nn.Module):
    def __init__(self, num_classes, image_size, channel):
        super(ConvNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(channel, 128, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(128, 128),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(128, 128),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            nn.GroupNorm(128, 128),
            nn.ReLU(inplace=True),
            nn.AvgPool2d(kernel_size=2, stride=2)
        )

        # 计算池化后的特征图大小
        reduced_size = image_size // (2 ** 3)  # 三次 avgpool each downsample by 2
        flattened_dim = 128 * reduced_size * reduced_size  # 特征图展开成一维

        self.classifier = nn.Linear(flattened_dim, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


def epoch(dataloader, net, optimizer, criterion, args):
    loss_avg, acc_avg, num_exp = 0, 0, 0
    net = net.to(args.device)
    criterion = criterion.to(args.device)

    net.train()

    for i_batch, datum in enumerate(dataloader):
        img = datum[0].float().to(args.device)
        lab = datum[1].long().to(args.device)
        n_b = lab.shape[0]

        output = net(img)
        loss = criterion(output, lab)
        acc = np.sum(np.equal(np.argmax(output.cpu().data.numpy(), axis=-1), lab.cpu().data.numpy()))

        loss_avg += loss.item() * n_b
        acc_avg += acc
        num_exp += n_b

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    loss_avg /= num_exp
    acc_avg /= num_exp

    return loss_avg, acc_avg


def test_acc(net, test_loader, args):
    net.eval()

    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(args.device), labels.to(args.device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy


def train(train_loader, test_loader, args):
    net = ConvNet(10, 28, 1).to(args.device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(net.parameters(), lr=args.lr_net, momentum=0.9, weight_decay=0.0005)

    for e in range(0, args.epochs + 1):
        if e % 10 == 0 or e == args.epochs:
            acc = test_acc(net, test_loader, args)
            print(f'{e} / {args.epochs}, test acc: {acc:.3f}')

        train_loss, train_acc = epoch(train_loader, net, optimizer, criterion, args)
        print(f'{e} / {args.epochs}, train_loss: {train_loss:.3f}, train_acc: {train_acc:.3f}')

    return acc


def hold_out(ratio, args):
    mean = [0.1307]
    std = [0.3081]
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
    dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
    dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

    full_dataset = torch.utils.data.ConcatDataset([dst_train, dst_test])

    targets = []
    for d in full_dataset.datasets:
        targets.extend(d.targets.numpy() if hasattr(d.targets, 'numpy') else d.targets)
    targets = np.array(targets)

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=ratio, random_state=42)
    indices = np.arange(len(full_dataset))
    train_idx, test_idx = next(splitter.split(indices, targets))

    train_subset = Subset(full_dataset, train_idx)
    test_subset = Subset(full_dataset, test_idx)

    train_loader = DataLoader(train_subset, batch_size=args.batch_train, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=args.batch_train, shuffle=False)

    return train_loader, test_loader


def k_fold(k, args):
    mean = [0.1307]
    std = [0.3081]
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
    dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
    dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)
    full_dataset = torch.utils.data.ConcatDataset([dst_train, dst_test])

    targets = []
    for d in full_dataset.datasets:
        targets.extend(d.targets.numpy() if hasattr(d.targets, 'numpy') else d.targets)
    targets = np.array(targets)

    indices = np.arange(len(full_dataset))

    skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

    for train_idx, test_idx in skf.split(indices, targets):
        train_subset = Subset(full_dataset, train_idx)
        test_subset = Subset(full_dataset, test_idx)

        train_loader = DataLoader(train_subset, batch_size=args.batch_train, shuffle=True)
        test_loader = DataLoader(test_subset, batch_size=args.batch_train, shuffle=False)

        yield train_loader, test_loader


if __name__ == '__main__':
    mean = [0.1307]
    std = [0.3081]
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=mean, std=std)])
    dst_train = datasets.MNIST(data_path, train=True, download=True, transform=transform)
    dst_test = datasets.MNIST(data_path, train=False, download=True, transform=transform)

    train_loader = DataLoader(dst_train, batch_size=args.batch_train, shuffle=True)
    test_loader = DataLoader(dst_test, batch_size=args.batch_train, shuffle=False)

    result_list = []
    for e in [1, 5, 10, 20]:
        for lr in [0.01, 0.1, 1.0]:
            for batch_size in [128, 256, 512]:
                args.epochs = e
                args.lr_net = lr
                args.batch_train = batch_size
                acc = train(train_loader, test_loader, args)
                result_list.append(f'epochs = {e}, lr = {lr}, batch_size = {batch_size}, acc = {acc:.3f}')
    print(result_list)

    for ratio in [0.1, 0.2, 0.3, 0.4, 0.5]:
        print(f'ratio = {ratio}')
        train_loader, test_loader = hold_out(ratio, args)
        acc = train(train_loader, test_loader, args)
        result_list.append(f'{ratio}: {acc:.3f}')
    print(result_list)

    for k in [5, 10, 20]:
        buf = []
        for train_loader, test_loader in k_fold(k, args):
            acc = train(train_loader, test_loader, args)
            buf.append(acc)
        mean, std = np.mean(buf), np.std(buf)
        result_list.append([k, mean, std])

    print(result_list)


usage: colab_kernel_launcher.py [-h] [--device DEVICE] [--lr_net LR_NET]
                                [--batch_train BATCH_TRAIN] [--epochs EPOCHS]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-2e196961-3e0a-428f-8412-8e1abf8bbe5b.json


SystemExit: 2